<a href="https://colab.research.google.com/github/hellizer4u/GenAi-Task04/blob/main/PRODIGY_GA_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time


In [ ]:
# Install TensorFlow (usually pre-installed in Colab)
!pip install tensorflow

# Install TensorFlow Datasets (optional for info)
!pip install tensorflow-datasets


In [ ]:
!wget http://efrosgans.eecs.berkeley.edu/pix2pix/datasets/facades.tar.gz
!tar -xvzf facades.tar.gz


In [ ]:
import tensorflow as tf
import numpy as np
from glob import glob
from PIL import Image

# Image loading & preprocessing function
def load_image(filename):
    image = Image.open(filename)
    image = image.resize((256, 256))  # Resize to 256x256
    image = np.array(image).astype(np.float32)
    image = (image / 127.5) - 1  # Normalize to [-1, 1]

    w = image.shape[1]
    w_half = w // 2

    input_image = image[:, :w_half, :]
    target_image = image[:, w_half:, :]
    return input_image, target_image

# TensorFlow wrapper for loading images
def tf_load_image(path):
    input_img, target_img = tf.numpy_function(load_image, [path], [tf.float32, tf.float32])
    input_img.set_shape([256, 128, 3])
    target_img.set_shape([256, 128, 3])
    return input_img, target_img


In [ ]:
# Get list of training image paths
train_paths = glob('facades/train/*.jpg')

# Create tf.data.Dataset from paths
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)

# Map preprocessing function and batch
train_dataset = train_dataset.map(tf_load_image, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.shuffle(buffer_size=400).batch(1).prefetch(tf.data.AUTOTUNE)


In [ ]:
from tensorflow.keras import layers

def downsample(filters, size, apply_batchnorm=True):
    initializer = tf.random_normal_initializer(0., 0.02)
    result = tf.keras.Sequential()
    result.add(layers.Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=False))
    if apply_batchnorm:
        result.add(layers.BatchNormalization())
    result.add(layers.LeakyReLU())
    return result

def upsample(filters, size, apply_dropout=False):
    initializer = tf.random_normal_initializer(0., 0.02)
    result = tf.keras.Sequential()
    result.add(layers.Conv2DTranspose(filters, size, strides=2, padding='same',
                                      kernel_initializer=initializer, use_bias=False))
    result.add(layers.BatchNormalization())
    if apply_dropout:
        result.add(layers.Dropout(0.5))
    result.add(layers.ReLU())
    return result

def Generator():
    inputs = layers.Input(shape=[256, 128, 3])

    down_stack = [
        downsample(64, 4, apply_batchnorm=False),  # (128, 64, 64)
        downsample(128, 4),  # (64, 32, 128)
        downsample(256, 4),  # (32, 16, 256)
        downsample(512, 4),  # (16, 8, 512)
        downsample(512, 4),  # (8, 4, 512)
        downsample(512, 4),  # (4, 2, 512)
        downsample(512, 4),  # (2, 1, 512)
        downsample(512, 4),  # (1, 1, 512)
    ]

    up_stack = [
        upsample(512, 4, apply_dropout=True),  # (2, 2, 512)
        upsample(512, 4, apply_dropout=True),  # (4, 4, 512)
        upsample(512, 4, apply_dropout=True),  # (8, 8, 512)
        upsample(512, 4),  # (16, 16, 512)
        upsample(256, 4),  # (32, 32, 256)
        upsample(128, 4),  # (64, 64, 128)
        upsample(64, 4),   # (128, 128, 64)
    ]

    initializer = tf.random_normal_initializer(0., 0.02)
    last = layers.Conv2DTranspose(3, 4, strides=2, padding='same',
                                  kernel_initializer=initializer, activation='tanh')  # (256, 256, 3)

    x = inputs
    skips = []
    for down in down_stack:
        x = down(x)
        skips.append(x)
    skips = reversed(skips[:-1])

    for up, skip in zip(up_stack, skips):
        x = up(x)
        x = layers.Concatenate()([x, skip])

    x = last(x)

    return tf.keras.Model(inputs=inputs, outputs=x)

def Discriminator():
    initializer = tf.random_normal_initializer(0., 0.02)

    inp = layers.Input(shape=[256, 128, 3], name='input_image')
    tar = layers.Input(shape=[256, 128, 3], name='target_image')

    x = layers.concatenate([inp, tar])  # concatenate input and target images

    down1 = downsample(64, 4, False)(x)  # (128, 64, 64)
    down2 = downsample(128, 4)(down1)    # (64, 32, 128)
    down3 = downsample(256, 4)(down2)    # (32, 16, 256)

    zero_pad1 = layers.ZeroPadding2D()(down3)  # (34, 18, 256)
    conv = layers.Conv2D(512, 4, strides=1,
                         kernel_initializer=initializer,
                         use_bias=False)(zero_pad1)  # (31, 15, 512)

    batchnorm1 = layers.BatchNormalization()(conv)
    leaky_relu = layers.LeakyReLU()(batchnorm1)

    zero_pad2 = layers.ZeroPadding2D()(leaky_relu)  # (33, 17, 512)

    last = layers.Conv2D(1, 4, strides=1,
                         kernel_initializer=initializer)(zero_pad2)  # (30, 14, 1)

    return tf.keras.Model(inputs=[inp, tar], outputs=last)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

def downsample(filters, size, apply_batchnorm=True):
    initializer = tf.random_normal_initializer(0., 0.02)
    result = tf.keras.Sequential()
    result.add(layers.Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=False))
    if apply_batchnorm:
        result.add(layers.BatchNormalization())
    result.add(layers.LeakyReLU())
    return result

def upsample(filters, size, apply_dropout=False):
    initializer = tf.random_normal_initializer(0., 0.02)
    result = tf.keras.Sequential()
    result.add(layers.Conv2DTranspose(filters, size, strides=2,
                                      padding='same',
                                      kernel_initializer=initializer,
                                      use_bias=False))
    result.add(layers.BatchNormalization())
    if apply_dropout:
        result.add(layers.Dropout(0.5))
    result.add(layers.ReLU())
    return result

class Generator(tf.keras.Model):
    def __init__(self):
        super(Generator, self).__init__()
        self.down1 = downsample(64, 4, apply_batchnorm=False)  # (bs, 128, 128, 64)
        self.down2 = downsample(128, 4)                        # (bs, 64, 64, 128)
        self.down3 = downsample(256, 4)                        # (bs, 32, 32, 256)
        self.down4 = downsample(512, 4)                        # (bs, 16, 16, 512)
        self.down5 = downsample(512, 4)                        # (bs, 8, 8, 512)
        self.down6 = downsample(512, 4)                        # (bs, 4, 4, 512)
        self.down7 = downsample(512, 4)                        # (bs, 2, 2, 512)
        self.down8 = downsample(512, 4, apply_batchnorm=False) # (bs, 1, 1, 512)

        self.up1 = upsample(512, 4, apply_dropout=True)        # (bs, 2, 2, 512)
        self.up2 = upsample(512, 4, apply_dropout=True)        # (bs, 4, 4, 512)
        self.up3 = upsample(512, 4, apply_dropout=True)        # (bs, 8, 8, 512)
        self.up4 = upsample(512, 4)                            # (bs, 16, 16, 512)
        self.up5 = upsample(256, 4)                            # (bs, 32, 32, 256)
        self.up6 = upsample(128, 4)                            # (bs, 64, 64, 128)
        self.up7 = upsample(64, 4)                             # (bs, 128, 128, 64)

        self.last = layers.Conv2DTranspose(3, 4, strides=2,
                                           padding='same',
                                           kernel_initializer=tf.random_normal_initializer(0., 0.02),
                                           activation='tanh')  # (bs, 256, 256, 3)

    def call(self, x):
        # Downsampling through the model
        d1 = self.down1(x)
        d2 = self.down2(d1)
        d3 = self.down3(d2)
        d4 = self.down4(d3)
        d5 = self.down5(d4)
        d6 = self.down6(d5)
        d7 = self.down7(d6)
        d8 = self.down8(d7)

        # Upsampling and establishing the skip connections
        u1 = self.up1(d8)
        u1 = tf.concat([u1, d7], axis=-1)
        u2 = self.up2(u1)
        u2 = tf.concat([u2, d6], axis=-1)
        u3 = self.up3(u2)
        u3 = tf.concat([u3, d5], axis=-1)
        u4 = self.up4(u3)
        u4 = tf.concat([u4, d4], axis=-1)
        u5 = self.up5(u4)
        u5 = tf.concat([u5, d3], axis=-1)
        u6 = self.up6(u5)
        u6 = tf.concat([u6, d2], axis=-1)
        u7 = self.up7(u6)
        u7 = tf.concat([u7, d1], axis=-1)

        return self.last(u7)


In [ ]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(disc_real_output, disc_generated_output):
    real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)
    generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)
    total_disc_loss = real_loss + generated_loss
    return total_disc_loss

def generator_loss(disc_generated_output, gen_output, target):
    gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)
    # L1 loss for pix2pix
    l1_loss = tf.reduce_mean(tf.abs(target - gen_output))
    total_gen_loss = gan_loss + (100 * l1_loss)
    return total_gen_loss

generator = Generator()
discriminator = Discriminator()

generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)


In [ ]:
@tf.function
def train_step(input_image, target):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        gen_output = generator(input_image, training=True)

        disc_real_output = discriminator([input_image, target], training=True)
        disc_generated_output = discriminator([input_image, gen_output], training=True)

        gen_loss = generator_loss(disc_generated_output, gen_output, target)
        disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

    gradients_gen = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_disc = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_gen, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_disc, discriminator.trainable_variables))


In [ ]:
import time  # <--- add this line

EPOCHS = 60

for epoch in range(EPOCHS):
    start = time.time()

    for input_image, target in train_dataset:
        train_step(input_image, target)

    print(f'Epoch {epoch + 1} completed in {time.time() - start:.2f} seconds')


In [ ]:
def load_image(filename):
    image = Image.open(filename)
    image = image.resize((512, 256))  # width=512, height=256
    image = np.array(image).astype(np.float32)
    image = (image / 127.5) - 1

    w = image.shape[1]
    w_half = w // 2

    input_image = image[:, :w_half, :]
    target_image = image[:, w_half:, :]

    # Resize to square 256x256
    input_image = tf.image.resize(input_image, [256, 256])
    target_image = tf.image.resize(target_image, [256, 256])

    return input_image, target_image


In [ ]:
inputs = layers.Input(shape=[256, 256, 3])


In [ ]:
def load_image(filename):
    image = Image.open(filename)
    image = image.resize((286, 256))  # resize slightly bigger for cropping

    image = np.array(image).astype(np.float32)
    image = (image / 127.5) - 1  # Normalize to [-1, 1]

    w = image.shape[1]
    w_half = w // 2

    input_image = image[:, :w_half, :]
    target_image = image[:, w_half:, :]

    input_image = tf.image.resize(input_image, [256, 256])
    target_image = tf.image.resize(target_image, [256, 256])

    return input_image, target_image

def tf_load_image(path):
    input_img, target_img = tf.numpy_function(load_image, [path], [tf.float32, tf.float32])
    input_img.set_shape([256, 256, 3])
    target_img.set_shape([256, 256, 3])
    return input_img, target_img


In [ ]:
def Generator():
    inputs = tf.keras.layers.Input(shape=[256, 256, 3])
    ...


In [ ]:
# Create tf.data.Dataset from paths again
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(tf_load_image, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.shuffle(buffer_size=400).batch(1).prefetch(tf.data.AUTOTUNE)


In [ ]:
for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1} started")
    start = time.time()

    for step, (input_image, target) in enumerate(train_dataset):
        train_step(input_image, target)

        if step % 100 == 0:
            print(f"Epoch {epoch+1} Batch {step} done")

    print(f"Time taken for epoch {epoch+1} is {time.time() - start:.2f} sec")

    generate_images(generator, input_image, target)


In [ ]:
import matplotlib.pyplot as plt

def generate_images(model, input_image, target):
    prediction = model(input_image, training=True)
    prediction = (prediction + 1) / 2.0  # rescale from [-1, 1] to [0, 1]
    input_image = (input_image + 1) / 2.0
    target = (target + 1) / 2.0

    plt.figure(figsize=(15, 5))

    display_list = [input_image[0], target[0], prediction[0]]
    title = ['Input Image', 'Target Image', 'Predicted Image']

    for i in range(3):
        plt.subplot(1, 3, i+1)
        plt.title(title[i])
        plt.imshow(display_list[i])
        plt.axis('off')
    plt.show()
